In [114]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from math import sqrt, pi, sin, cos, tan
#from __future__ import division

img = cv2.imread('img8.jpg')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

list1 = []      # separate lists to store the coordinates for the separate lines
list2 = []
check = 0       # variable to switch between the lists

def getPoints(event, x, y, flags, params): # get Parallel Lines
    
    global list1, list2, check    
   
    if event == cv2.EVENT_LBUTTONDOWN:
        print (x,y)
        if check < 4:
            list1.append([x, y])
            check += 1
        else:
            list2.append([x, y])
    elif event == cv2.EVENT_LBUTTONUP:                   
        cv2.circle(img,(x,y), 4, (0,255,0), -1)
        cv2.imshow("image", img)

    
cv2.namedWindow("image")
cv2.setMouseCallback("image", getPoints)

def selectPoints():
    global check
    while True:
        cv2.imshow("image", img)
        key = cv2.waitKey(1) & 0xFF        
        
        if key == ord("q"):
            cv2.imwrite("PointsSelected.jpg", img)
            break
#         elif check == 7:
#             break
        
selectPoints()
cv2.destroyAllWindows()
cv2.waitKey(1)

278 190
294 222
304 187
325 215
210 189
234 196
188 215
220 225


-1

In [115]:
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

In [116]:
print (list1, list2)

# This function returns the vanishing line equation
def vanishingline(list1, list2):
    
    # Equation of lines
    l1 = np.cross([list1[0][0], list1[0][1], 1], [list1[1][0], list1[1][1], 1])
    l2 = np.cross([list1[2][0], list1[2][1], 1], [list1[3][0], list1[3][1], 1])
    p1 = np.cross([list2[0][0], list2[0][1], 1], [list2[1][0], list2[1][1], 1])
    p2 = np.cross([list2[2][0], list2[2][1], 1], [list2[3][0], list2[3][1], 1])
    
    temp1 = np.cross(l1,l2)
    temp2 = np.cross(p1,p2)
    
    # Vanishing Points
    v1 = temp1/temp1[2]
    v2 = temp2/temp2[2]
    print (v1, v2)
    
    return np.cross(v1, v2)
    

vLine = vanishingline(list1, list2)


# Returns the Affine Rectification Homography Matrix (that represents projective distortion)
def affineHom(line):    
    return np.array([[1, 0, 0], [0, 1, 0], [line[0]/line[2], line[1]/line[2], 1]])


HomMatrix = np.float32(affineHom(vLine))

# # M = cv2.getPerspectiveTransform(pts1,pts2)
size = gray.shape
sizeNew = (size[1], size[0])
AffineRect = cv2.warpPerspective(img,HomMatrix, sizeNew)
#cv2.imshow("Affine Rectification", AffineRect)
cv2.imwrite("AffineRectifiedImage.jpg", AffineRect)
cv2.waitKey(500)

[[278, 190], [294, 222], [304, 187], [325, 215]] [[210, 189], [234, 196], [188, 215], [220, 225]]
[221.5  77.    1. ] [-1.3680e+03 -2.7125e+02  1.0000e+00]


-1

In [92]:
img = AffineRect
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
 
# These two lists keep the points for perpendicular lines calculation
Mlist1 = []
Mlist2 = []
checkM = 0

def getPointsMetric(event, x, y, flags, params):
    
    global Mlist1, Mlist2, checkM 
   
    if event == cv2.EVENT_LBUTTONDOWN:
        print (x,y)
        if checkM < 3:
            Mlist1.append([x, y])
            checkM += 1
        else:
            Mlist2.append([x, y])
    elif event == cv2.EVENT_LBUTTONUP:                   
        cv2.circle(img,(x,y), 4, (0,255,0), -1)
        cv2.imshow("image", img)

    
cv2.namedWindow("image")
cv2.setMouseCallback("image", getPointsMetric)

selectPoints()
cv2.destroyAllWindows()
cv2.waitKey(1)

# This function returns the perpendicular lines required for metric rectification
def returnPerLines(list1, list2):
    
    # Equation of lines
    l1 = np.cross([list1[0][0], list1[0][1], 1], [list1[1][0], list1[1][1], 1])
    m1 = np.cross([list1[2][0], list1[2][1], 1], [list1[1][0], list1[1][1], 1])
    
    l2 = np.cross([list2[0][0], list2[0][1], 1], [list2[1][0], list2[1][1], 1])
    m2 = np.cross([list2[2][0], list2[2][1], 1], [list2[1][0], list2[1][1], 1])

    return l1/l1[2], m1/m1[2], l2/l2[2], m2/m2[2]
    

l1, m1, l2, m2 = returnPerLines(Mlist1, Mlist2)



380 33
307 38
341 66
345 16
333 9
253 20


In [93]:
# This function returns the symmetric matrix
def symmetricMatrix(l1, m1, l2, m2):
    C = np.array([-l1[1]*m1[1], -l2[1]*m2[1]])
    A = np.array([[l1[0]*m1[0], l1[0]*m1[1] + l1[1]*m1[0]], [l2[0]*m2[0], l2[0]*m2[1] + l2[1]*m2[0]]])
    
    s = np.matmul(np.linalg.inv(A), C)
    
    # symmetric matrix
    sWhole = np.array([[s[0], s[1]], [s[1], 1]])
    return sWhole
    
smatrix = symmetricMatrix(l1, m1, l2, m2)

U, D, V = np.linalg.svd(smatrix)
print (U, D, V)



[[-0.9962978 -0.0859691]
 [-0.0859691  0.9962978]] [8.78946846 0.94200183] [[-0.9962978 -0.0859691]
 [-0.0859691  0.9962978]]


In [94]:
# This function returns the affine homography matrix
def affineHom(U, D, V):
    Dtemp = np.sqrt(D)
    Dfinal = np.array([[Dtemp[0], 0], [0, Dtemp[1]]])
    
    # The final A matrix
    Am = np.matmul(np.matmul(U, Dfinal), V)
     
    print (np.array([[Am[0][0], Am[0][1], 0], [Am[1][0], Am[1][1], 0]]))
    return np.array([[Am[0][0], Am[0][1], 0], [Am[1][0], Am[1][1], 0], [0, 0, 1]])
    
    
HomAffine = affineHom(U, D, V)
InvHomeAffine = np.float32(np.linalg.inv(HomAffine))  

MetricRect = cv2.warpPerspective(AffineRect,InvHomeAffine, sizeNew)

#cv2.imshow("Metric Rectification", MetricRect)
cv2.imwrite("MetricRectifiedImage.jpg", MetricRect)
cv2.waitKey(500)

[[2.94996574 0.1707994  0.        ]
 [0.1707994  0.98530582 0.        ]]


-1

In [ ]:
cv2.destroyAllWindows()
cv2.waitKey(1)